In [11]:
#reference : https://www.zenrows.com/blog/dynamic-web-pages-scraping-python#selecting-elements
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import re


In [12]:
trainers_data = pd.read_csv('c:\\Users\\IM_py\\Documents\\GitHub\\Projects\\Webscrapping-lesson-with-BeautifulSoup\\data\\trainers_premiership.csv')
trainers_data.count()

Rank       260
Trainer    260
Wins       260
2nds       260
3rds       260
Stakes     260
Starts     260
G&L        260
SR         260
dtype: int64

In [13]:
names_for_url = trainers_data['Trainer'].str.lower().str.replace("_","-") + '-nz'

In [14]:
names_for_url = names_for_url[~names_for_url.str.contains('&')]

In [43]:
def safe_get(lst, idx, default=''):
    """Safely access the item at `idx` in the list `lst`, returning `default` if out of range."""
    try:
        return lst[idx]
    except IndexError:
        return default
    

def parse_entry(entry):
    lines = entry.split('\n')
    return {
        "Position": safe_get(lines, 0),
        "Horse": safe_get(lines, 1),
        "Trainer": safe_get(lines, 3).strip('T: '),
        "Jockey": safe_get(lines, 5).strip('J: '),
        "Track": safe_get(lines, 6),
        "Date": safe_get(lines, 7),
        "Distance": safe_get(lines, 8),
        "Condition": safe_get(lines, 9),
        "Prize": safe_get(lines, 10),
        "Class": safe_get(lines, 11),
        "WGT": safe_get(lines, 12),
        "Pos_800_400": safe_get(lines, 13),
        "Margin": safe_get(lines, 14),
        "Rating": safe_get(lines, 15),
        "Odds": safe_get(lines, 16)
    }

In [65]:
options = Options()
options.headless = True
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

for current_trainer in names_for_url:

    url = 'https://www.racing.com/trainers/'+current_trainer+'/form'
    driver.get(url)
    print(current_trainer)
    time.sleep(5)
    print(driver.title)

    if '404' in driver.title:
        print(f"Page not found for trainer: {current_trainer}")
        continue 

    try:
        WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.TAG_NAME, "tr")) 
        )
        time.sleep(5)
        elements = driver.find_elements(By.TAG_NAME, 'table') 
        trainers_form_races = [trainers_elements.text for trainers_elements in elements]
    finally:
        trainers_form_races = trainers_form_races if 'trainers_form_races' in locals() else [""]
        print(trainers_form_races) 
    # Split the data into individual entries
    # Assuming each entry starts with a pattern like '7-18' followed by a horse name
    entries = re.split(r'\n(?=\d+-\d+\n)', trainers_form_races[0])[1:]  # [1:] to skip the header

    # Parse each entry
    parsed_data = [parse_entry(entry) for entry in entries]

    # Create DataFrame
    df = pd.DataFrame(parsed_data)

    print(df)  
    df.to_csv('c:\\Users\\mypath'+current_trainer+'.csv',index=False)
    

cody-cole-nz
404 Not Found - RACING.COM
Page not found for trainer: cody-cole-nz
kelvin-tyler-nz
404 Not Found - RACING.COM
Page not found for trainer: kelvin-tyler-nz
team-rogerson-nz
404 Not Found - RACING.COM
Page not found for trainer: team-rogerson-nz
kenny-moore-nz
404 Not Found - RACING.COM
Page not found for trainer: kenny-moore-nz
john-bary-nz
John Bary - Trainer Profile - RACING.COM
["POS\nHORSE/TRAINER/JOCKEY\nTRACK/DATE\nDIST/COND\nPRIZE/CLASS\nWGT\n800M/400M\nMARGIN\nRATING\nODDS\nCOM\n7-8\n4. Callsign Mav (NZ) (4)\nT: \nJ.Bary\n  J: \nL.Nolen\nM V R9\n23 OCT 21\n2040M\nSoft 5\n$5M\nG1 COX PLATE\n59kg\n2nd / 5th\n13.85L\n115\n$31\nSP\n1-12\nCallsign Mav (NZ) (7)\nT: \nJ.Bary\n  J: \nJ.K.Riddell\nHAST R8\n02 OCT 21\n1600M\nGood\n$205.9K\nG1 WINDSOR PARK\n59kg\n-\n-\n-\n$1.8\nSP\n1-13\nCallsign Mav (NZ) (10)\nT: \nJ.Bary\n  J: \nJ.K.Riddell\nHAST R9\n18 SEP 21\n1400M\nHeavy\n$205.9K\nG1 TARZINO\n59kg\n-\n-\n-\n$3.4\nSP\n3-13\nCallsign Mav (NZ) (7)\nT: \nJ.Bary\n  J: \nJ.K.Ri

we have the names of the trainers, now we can loop trough them, access the page of each and get their info, but before that, let's define the table making roadmap:

- Split the Data: First, split the string into individual entries based on a unique pattern. In this case, it seems each entry starts with a position like "7-18".

- Parse Each Entry: For each entry, further split the information into its respective columns. This might involve identifying specific patterns or separators in the text.

- Create a Data Structure: Store the parsed data in a structured format like a list of dictionaries or a list of lists, which can then be easily converted into a DataFrame.

- Convert to DataFrame: Use pandas to convert this structured data into a DataFrame for easier manipulation and analysis.

In [ ]:
'https://www.racing.com/trainers/'+names_for_url[1]+'/form'

In [ ]:
trainers_form_races[0].find('-')

## What about that info on the top right? Looks interesting right?!

 Buts it's harder to get by tagname for example, so we can use it's path. 

In [ ]:
elements_topright = driver.find_elements(By.XPATH, '//*[@id="skin-wrapper"]/div/div[2]/div[3]') 
topright_stats = [trainers_elements.text for trainers_elements in elements_topright]
topright_stats


In [ ]:
tr_list = topright_stats[0].split('\n')
column = [''.join(tr_list[x].lower().split()) for x in range(0,len(tr_list),3)]
data = [tr_list[x] for x in range(1,len(tr_list),3)]

In [ ]:
trainer_stats_df = pd.DataFrame(data = [[tr_list[x] for x in range(1,len(tr_list),3)]],
             columns=[tr_list[x].lower() for x in range(0,len(tr_list),3)])

trainer_stats_df['trainer'] = 'fulano'
trainer_stats_df

In [ ]:
#''.join('TES ET'.lower().split())
column